# SNMP - Simple Network Management Protocol

## Install snmpd tools, agent and mibs

In [10]:
%%bash

# Install SNMP - Linux
sudo apt update
sudo apt install -y snmp snmpd snmp-mibs-downloader

Hit:1 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:2 https://dl.yarnpkg.com/debian stable InRelease


Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Reading package lists...
Building dependency tree...
Reading state information...
32 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
snmp-mibs-downloader is already the newest version (1.2).
snmp is already the newest version (5.8+dfsg-2ubuntu2.9).
snmpd is already the newest version (5.8+dfsg-2ubuntu2.9).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


## Configure agent configuration file (/etc/snmp/snmpd.conf) and snmp tools configuration file (/etc/snmp/snmp.conf)

In [ ]:
%%bash

sudo mv /etc/snmp/snmpd.conf /etc/snmp/snmpd.conf.old

sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private
EOF

sudo sed -i '/mibs :/s/^/#/' /etc/snmp/snmp.conf

## Start snmpd daemon (agent)

In [ ]:
%%bash

sudo service snmpd restart
sudo service snmpd status

# Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

## List all downloaded mibs

In [ ]:
%%bash

ls -R /usr/share/snmp/mibs

## Show RFC1213 object tree

In [ ]:
%%bash

# Variables
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

snmptranslate -m $MIB -Tp

## Show the description of a specific object

In [ ]:
%%bash

# Variables
OBJECT="sysDescr"
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

# Command to extract text from search_string to } excluding the }
sed -n "/$OBJECT OBJECT-TYPE/,/}/p" $MIB

## Using snmpget

In [ ]:
%%bash

# snmpget -v <SNMP version> -c <community> <host> <obj1> <obj2> <obj3> 

snmpget -v 1 -c public localhost sysDescr.0 sysContact.0

## Using snmpset

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysContact.0 s admin@localhost
snmpget -v 1 -c public localhost sysContact.0

## Using snmpgetnext

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost system interfaces

## Using snmpwalk

In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
snmpwalk -v 1 -c public localhost system

## Other commands

In [ ]:
%%bash

snmptable -v 1 -c public localhost ifTable

In [ ]:
%%bash

snmpdelta -v 1 -c public -Cs -CT localhost sysUpTime.0

In [ ]:
%%bash

killall snmpdelta

## Exercícios - SNMPv1

a) obter o endereço físico (MAC) da 2a. interface de rede da tabela ifTable

In [ ]:
%%bash

snmpget -v 1 -c public localhost ifPhysAddress.2

b) obter o número de mensagens ICMP enviadas e recebidas pelo elemento gerenciado

In [ ]:
%%bash

snmpget -v 1 -c public localhost icmpInMsgs.0 icmpOutMsgs.0

c) obter o número de requisições GET recebidos pelo agente no elemento gerenciado.

In [ ]:
%%bash

snmpget -v 1 -c public localhost snmpInGetRequests.0

d) modificar o nome do elemento gerenciado para "Gerencia"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysName.0 s "Gerencia"

e) modificar a identificação da localização do elemento gerenciado para "LabRedes"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysLocation.0 s "LabRedes"

f) obter a descrição da interface localizada na 1a. linha da tabela ifTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ifDescr

g) obter o endereço IP, índice da interface na tabela ifTable, máscara de rede e endereço de broadcast da primeira entrada da tabela ipAddrEntry

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ipAdEntIfIndex ipAdEntNetMask ipAdEntBcastAddr

h) obter o endereço local e porta local da primeira linha da tabela udpTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost udpLocalPort udpLocalAddress

## Using snmpbulkget

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn2 -Cr3 system interfaces ifIndex ifDescr ifSpeed

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn4 -Cr5 tcpInSegs tcpOutSegs tcpRetransSegs tcpInErrs tcpConnState tcpConnLocalAddress tcpConnLocalPort tcpConnRemAddress tcpConnRemPort

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn0 -Cr4 udpLocalAddress udpLocalPort

## Comparing snmpwalk vs snmpbulkwalk



In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
# snmpwalk -v 1 -c public localhost system

snmpbulkwalk -v 2c -c public localhost system

# Example using pass directive

## snmpd.conf

In [3]:
%%bash

# Add pass directive to snmpd.conf
sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private

pass .1.3.6.1.3.1234.1 /usr/bin/python3 /tmp/agent.py
EOF

# Restart agent - Linux
sudo service snmpd restart
sudo service snmpd status

# Restart agent - Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

 * snmpd is running


## Agent code

In [7]:
#!/usr/bin/env python3

import sys
import json

# Caminho para o arquivo JSON com os dados do SmartWatch
json_file_path = './smartwatch_data.json'

def load_data():
    try:
        with open(json_file_path, 'r') as file:
            return json.load(file)
    except Exception as e:
        print("Failed to load JSON data:", e)
        sys.exit(1)

# Carregar os dados do SmartWatch no início do script
smartwatch_data = load_data()

def get_oid_value(oid):
    mappings = {
        "1.3.6.1.3.1234.1.1.0": ("string", smartwatch_data["status"]),
        "1.3.6.1.3.1234.1.2.0": ("string", smartwatch_data["name"]),
        "1.3.6.1.3.1234.1.3.0": ("integer", smartwatch_data["daily_steps"]),
        "1.3.6.1.3.1234.1.4.0": ("integer", smartwatch_data["battery_level"]),
        "1.3.6.1.3.1234.1.5.0": ("integer", smartwatch_data["heart_rate"]),
        "1.3.6.1.3.1234.1.6.0": ("integer", int(smartwatch_data["power_saving_mode"])),
        "1.3.6.1.3.1234.1.7.0": ("integer", smartwatch_data["calories_burned"]),
        "1.3.6.1.3.1234.1.8.0": ("integer", smartwatch_data["distance_traveled"]),
        "1.3.6.1.3.1234.1.9.0": ("integer", int(smartwatch_data["connected_to_smartphone"])),
        "1.3.6.1.3.1234.1.10.0": ("integer", smartwatch_data["notifications"]),
        "1.3.6.1.3.1234.1.11.0": ("integer", smartwatch_data["step_goal"]),
        "1.3.6.1.3.1234.1.12.0": ("integer", int(smartwatch_data["is_charging"]))
    }
    return mappings.get(oid)

def handle_get(oid):
    result = get_oid_value(oid)
    if result:
        return f"{oid}\n{result[0]}\n{result[1]}"
    return "NONE"

def handle_getnext(oid):
    oids = sorted(get_oid_value(None).keys())
    next_oid_index = oids.index(oid) + 1 if oid in oids else 0
    if next_oid_index < len(oids):
        return handle_get(oids[next_oid_index])
    return "NONE"

def main():
    if len(sys.argv) < 3:
        print("Usage: agent.py -g|-n|-s OID [type new_value]")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        print(handle_get(oid))
    elif request_type == "-n":  # GETNEXT request
        print(handle_getnext(oid))
    else:
        print("NONE")

if __name__ == "__main__":
    main()


Usage: agent.py -g|-n|-s OID [type new_value]


## Testing

In [8]:
%%bash

snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.1.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.2.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.3.0

iso.3.6.1.3.1234.1.1.0 = No Such Instance currently exists at this OID
iso.3.6.1.3.1234.1.2.0 = No Such Instance currently exists at this OID
iso.3.6.1.3.1234.1.3.0 = No Such Instance currently exists at this OID


In [ ]:
%%bash

snmpset -v2c -c private localhost .1.3.6.1.3.1234.1.3.0 s teste

In [ ]:
%%writefile MYMIB.txt

MYMIB DEFINITIONS ::= BEGIN

IMPORTS
    OBJECT-GROUP FROM SNMPv2-CONF
    OBJECT-TYPE, experimental FROM SNMPv2-SMI
    DisplayString FROM SNMPv2-TC;

myMIB MODULE-IDENTITY
    LAST-UPDATED "202406290000Z"
    ORGANIZATION "PUCRS"
    CONTACT-INFO "admin@localhost"
    DESCRIPTION "My MIB"
::= { experimental 1234 }

myObjects OBJECT IDENTIFIER ::= { myMIB 1 }

myCurrentDateTime OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Current Date and Time"
    ::= { myObjects 1 }

myHostname OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Hostname"
    ::= { myObjects 2 }

myFileContent OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-write
    STATUS current
    DESCRIPTION "File Content"
    ::= { myObjects 3 }

END

In [ ]:
%%bash

snmpget -v2c -c public -M +. -m +MYMIB localhost myCurrentDateTime.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myHostname.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myFileContent.0


In [ ]:
%%bash

snmpset -v2c -c private -M +. -m +MYMIB localhost myFileContent.0 s teste